[View in Colaboratory](https://colab.research.google.com/github/rabex028/machine_learning_skynet/blob/master/spamNeuralNetwork.ipynb)

##Setting up imports (including dataset)

In [0]:
import sys
import tensorflow as tf
import numpy as np
import argparse
import os
import shutil
import pandas as pd
!pip install gspread
import gspread
import io
from sklearn.model_selection import train_test_split


#Package for neural networks
!pip install -q keras
import keras
from keras.models import Sequential
from keras.layers import Dense

from google.colab import files
uploaded = files.upload()

spambase = pd.read_csv(io.StringIO(uploaded['spambase.csv'].decode('utf-8')), header = None)
spambase

##Initial Hypothesis

It doesn't look like there are hard and fast rules as to how many nodes should be in each layer, or how many layers there are, but a common starting point is an input layer with the number of features in each example, and the first hidden layer being half this amount. So our initial results will likely involve an input layer defined with 57 nodes to match the number of our features, and first hidden layer with 28 or so nodes. I think it might be better for our hidden layers to have a progressively smaller amount of nodes per level, which would mean that there will need to be aggregation of the results in previous layers. It might make sense then to go through the list of features and consider which ones may indicate an email is more likely to be spam and not spam (or neutral) and we can group our layers and nodes that way, and the result will be more effective.

##Creating and Running the Network (TensorFlow)

In [2]:
# Seperates independent and dependent variables
X = spambase.iloc[:,0:57].values
Y = spambase.iloc[:,57].values

# Uses sklearn module
# Creates training and test sets randomly with test set 20% of the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Defining Architecture 
# Only one hidden layer (for now)
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer
  
# Defining amounts of nodes, number of classifications
n_hidden_1 = 28
n_input = X_train.shape[1]
n_classes = 2

# Sets up weights and biases (connects nodes)
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder("float")

# 5000 training optimization runs, results every 1000, 32 rows loaded in each time
training_epochs = 5000
display_step = 1000
batch_size = 32

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder(tf.int64, [None])

predictions = multilayer_perceptron(x, weights, biases, keep_prob)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=predictions))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(X_train) / batch_size)
        x_batches = np.array_split(X_train, total_batch)
        y_batches = np.array_split(Y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test, keep_prob: 1.0}))

Epoch: 0001 cost= 291.721559475
Epoch: 1001 cost= 0.215419255
Epoch: 2001 cost= 0.166719687
Epoch: 3001 cost= 0.137684517
Epoch: 4001 cost= 0.137787282
Optimization Finished!
Accuracy: 0.9533116


##Creating and Running the Network (Keras)

In [3]:
# Initializing the Neural Network
spamClassifier = Sequential()

# Defining input layer and first hidden layer
# Number of nodes matches features, features/2
spamClassifier.add(Dense(units = 28, kernel_initializer = 'uniform', activation = 'relu', input_dim = 57))

# Defining 2nd layer
spamClassifier.add(Dense(units = 28, kernel_initializer = 'uniform', activation = 'relu'))

# Defining output layer
spamClassifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Setting hyperparameters
spamClassifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

spamClassifier.fit(X_train, Y_train, batch_size = 20, epochs = 100)

Epoch 1/100
3680/3680 [==============================] - 2s 457us/step - loss: 0.5850 - acc: 0.7465
Epoch 2/100
3680/3680 [==============================] - 1s 371us/step - loss: 0.4208 - acc: 0.8389
Epoch 3/100
3680/3680 [==============================] - 1s 373us/step - loss: 0.3132 - acc: 0.8837
Epoch 4/100
3680/3680 [==============================] - 1s 364us/step - loss: 0.2489 - acc: 0.9106
Epoch 5/100
3680/3680 [==============================] - 1s 374us/step - loss: 0.3061 - acc: 0.8984
Epoch 6/100
3680/3680 [==============================] - 1s 374us/step - loss: 0.2209 - acc: 0.9182
Epoch 7/100
3680/3680 [==============================] - 1s 376us/step - loss: 0.2217 - acc: 0.9190
Epoch 8/100
3680/3680 [==============================] - 1s 370us/step - loss: 0.2347 - acc: 0.9174
Epoch 9/100
 440/3680 [==>...........................] - ETA: 1s - loss: 0.2172 - acc: 0.9250

3680/3680 [==============================] - 1s 371us/step - loss: 0.2597 - acc: 0.9179
Epoch 10/100
3680/3680 [==============================] - 1s 369us/step - loss: 0.2514 - acc: 0.9084
Epoch 11/100
3680/3680 [==============================] - 1s 371us/step - loss: 0.2021 - acc: 0.9261
Epoch 12/100
3680/3680 [==============================] - 1s 362us/step - loss: 0.2326 - acc: 0.9201
Epoch 13/100
3680/3680 [==============================] - 1s 367us/step - loss: 0.2091 - acc: 0.9204
Epoch 14/100
3680/3680 [==============================] - 1s 377us/step - loss: 0.1847 - acc: 0.9323
Epoch 15/100
3680/3680 [==============================] - 1s 367us/step - loss: 0.2527 - acc: 0.9046
Epoch 16/100
3680/3680 [==============================] - 1s 372us/step - loss: 0.1969 - acc: 0.9285
Epoch 17/100
1160/3680 [========>.....................] - ETA: 0s - loss: 0.1761 - acc: 0.9371

3680/3680 [==============================] - 1s 373us/step - loss: 0.1884 - acc: 0.9296
Epoch 18/100
3680/3680 [==============================] - 1s 370us/step - loss: 0.1870 - acc: 0.9340
Epoch 19/100
3680/3680 [==============================] - 1s 378us/step - loss: 0.1768 - acc: 0.9359
Epoch 20/100
3680/3680 [==============================] - 1s 373us/step - loss: 0.1793 - acc: 0.9323
Epoch 21/100
3680/3680 [==============================] - 1s 366us/step - loss: 0.1782 - acc: 0.9342
Epoch 22/100
3680/3680 [==============================] - 1s 364us/step - loss: 0.1962 - acc: 0.9285
Epoch 23/100
3680/3680 [==============================] - 1s 365us/step - loss: 0.1873 - acc: 0.9310
Epoch 24/100
3680/3680 [==============================] - 1s 374us/step - loss: 0.1725 - acc: 0.9372
Epoch 25/100
1320/3680 [=========>....................] - ETA: 0s - loss: 0.1719 - acc: 0.9417

3680/3680 [==============================] - 1s 367us/step - loss: 0.1710 - acc: 0.9370
Epoch 26/100
3680/3680 [==============================] - 1s 364us/step - loss: 0.1896 - acc: 0.9307
Epoch 27/100
3680/3680 [==============================] - 1s 374us/step - loss: 0.2045 - acc: 0.9215
Epoch 28/100
3680/3680 [==============================] - 1s 374us/step - loss: 0.1880 - acc: 0.9302
Epoch 29/100
3680/3680 [==============================] - 1s 370us/step - loss: 0.1630 - acc: 0.9383
Epoch 30/100
3680/3680 [==============================] - 1s 367us/step - loss: 0.2061 - acc: 0.9231
Epoch 31/100
3680/3680 [==============================] - 1s 368us/step - loss: 0.1773 - acc: 0.9326
Epoch 32/100
3680/3680 [==============================] - 1s 375us/step - loss: 0.1623 - acc: 0.9408
Epoch 33/100
 880/3680 [======>.......................] - ETA: 1s - loss: 0.2701 - acc: 0.8920

3680/3680 [==============================] - 1s 375us/step - loss: 0.1827 - acc: 0.9321
Epoch 34/100
3680/3680 [==============================] - 1s 366us/step - loss: 0.1671 - acc: 0.9391
Epoch 35/100
3680/3680 [==============================] - 1s 362us/step - loss: 0.1736 - acc: 0.9345
Epoch 36/100
3680/3680 [==============================] - 1s 365us/step - loss: 0.1534 - acc: 0.9437
Epoch 37/100
3680/3680 [==============================] - 1s 376us/step - loss: 0.1527 - acc: 0.9421
Epoch 38/100
3680/3680 [==============================] - 1s 366us/step - loss: 0.1576 - acc: 0.9413
Epoch 39/100
3680/3680 [==============================] - 1s 367us/step - loss: 0.1630 - acc: 0.9416
Epoch 40/100
3680/3680 [==============================] - 1s 369us/step - loss: 0.1560 - acc: 0.9440
Epoch 41/100
1060/3680 [=======>......................] - ETA: 0s - loss: 0.1645 - acc: 0.9292

...

3680/3680 [==============================] - 1s 364us/step - loss: 0.1764 - acc: 0.9342
Epoch 66/100
3680/3680 [==============================] - 1s 365us/step - loss: 0.1442 - acc: 0.9457
Epoch 67/100
3680/3680 [==============================] - 1s 370us/step - loss: 0.1465 - acc: 0.9459
Epoch 68/100
3680/3680 [==============================] - 1s 373us/step - loss: 0.1441 - acc: 0.9484
Epoch 69/100
3680/3680 [==============================] - 1s 374us/step - loss: 0.1413 - acc: 0.9481
Epoch 70/100
3680/3680 [==============================] - 1s 378us/step - loss: 0.1315 - acc: 0.9519
Epoch 71/100
3680/3680 [==============================] - 1s 369us/step - loss: 0.1305 - acc: 0.9497
Epoch 72/100
3680/3680 [==============================] - 1s 366us/step - loss: 0.1375 - acc: 0.9478
Epoch 73/100
1020/3680 [=======>......................] - ETA: 1s - loss: 0.1384 - acc: 0.9520

3680/3680 [==============================] - 1s 374us/step - loss: 0.1471 - acc: 0.9459
Epoch 74/100
3680/3680 [==============================] - 1s 365us/step - loss: 0.1330 - acc: 0.9505
Epoch 75/100
3680/3680 [==============================] - 1s 367us/step - loss: 0.1272 - acc: 0.9535
Epoch 76/100
3680/3680 [==============================] - 1s 366us/step - loss: 0.1351 - acc: 0.9505
Epoch 77/100
3680/3680 [==============================] - 1s 364us/step - loss: 0.1244 - acc: 0.9535
Epoch 78/100
3680/3680 [==============================] - 1s 371us/step - loss: 0.1229 - acc: 0.9546
Epoch 79/100
3680/3680 [==============================] - 1s 372us/step - loss: 0.1311 - acc: 0.9543
Epoch 80/100
3680/3680 [==============================] - 1s 373us/step - loss: 0.1277 - acc: 0.9552
Epoch 81/100
1200/3680 [========>.....................] - ETA: 0s - loss: 0.1118 - acc: 0.9550

3680/3680 [==============================] - 1s 368us/step - loss: 0.1175 - acc: 0.9584
Epoch 82/100
3680/3680 [==============================] - 1s 371us/step - loss: 0.1222 - acc: 0.9562
Epoch 83/100
3680/3680 [==============================] - 1s 377us/step - loss: 0.1177 - acc: 0.9535
Epoch 84/100
3680/3680 [==============================] - 1s 378us/step - loss: 0.1107 - acc: 0.9587
Epoch 85/100
3680/3680 [==============================] - 1s 378us/step - loss: 0.1193 - acc: 0.9560
Epoch 86/100
3680/3680 [==============================] - 1s 372us/step - loss: 0.1142 - acc: 0.9601
Epoch 87/100
3680/3680 [==============================] - 1s 381us/step - loss: 0.1205 - acc: 0.9565
Epoch 88/100
3680/3680 [==============================] - 1s 377us/step - loss: 0.1234 - acc: 0.9571
Epoch 89/100
 700/3680 [====>.........................] - ETA: 1s - loss: 0.2216 - acc: 0.9171

3680/3680 [==============================] - 1s 378us/step - loss: 0.1577 - acc: 0.9402
Epoch 90/100
3680/3680 [==============================] - 1s 381us/step - loss: 0.1310 - acc: 0.9519
Epoch 91/100
3680/3680 [==============================] - 1s 380us/step - loss: 0.1180 - acc: 0.9565
Epoch 92/100
3680/3680 [==============================] - 1s 386us/step - loss: 0.1171 - acc: 0.9562
Epoch 93/100
3680/3680 [==============================] - 1s 365us/step - loss: 0.1191 - acc: 0.9565
Epoch 94/100
3680/3680 [==============================] - 1s 378us/step - loss: 0.1168 - acc: 0.9595
Epoch 95/100
3680/3680 [==============================] - 1s 367us/step - loss: 0.1149 - acc: 0.9571
Epoch 96/100
3680/3680 [==============================] - 1s 369us/step - loss: 0.1098 - acc: 0.9606
Epoch 97/100
 740/3680 [=====>........................] - ETA: 1s - loss: 0.1252 - acc: 0.9473

3680/3680 [==============================] - 1s 365us/step - loss: 0.1155 - acc: 0.9549
Epoch 98/100
3680/3680 [==============================] - 1s 375us/step - loss: 0.1122 - acc: 0.9595
Epoch 99/100
3680/3680 [==============================] - 1s 370us/step - loss: 0.1036 - acc: 0.9630
Epoch 100/100
3680/3680 [==============================] - 1s 373us/step - loss: 0.1082 - acc: 0.9598


##Resources:</br>
https://towardsdatascience.com/neural-networks-with-google-colaboratory-artificial-intelligence-getting-started-713b5eb07f14 </br>
https://medium.com/@curiousily/tensorflow-for-hackers-part-ii-building-simple-neural-network-2d6779d2f91b</br>
https://www.analyticsvidhya.com/blog/2016/10/an-introduction-to-implementing-neural-networks-using-tensorflow/